In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import plasma.parallel_processing as pp
import time

/u02/thanhhm/research-idea/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(data):
    time.sleep(1)
    return data * 2

def print_pass(data):
    print(f'{data / 2**5}')
    return data

In [4]:
outputs = pp.queues.ThreadQueue()
class Outer(pp.communicators.Flow):

    def __init__(self):
        super().__init__()

        self.compute2 = compute
        self.printer = print_pass
        self.aggregator = pp.communicators.Aggregator(20)

        self.registerIOs(
            compute2=pp.queues.ProcessQueue(2),
            printer=pp.queues.ProcessQueue(),
            aggregator=pp.queues.TransferQueue(),
            outputs=outputs
        )

caller = Outer()
caller

/u02/thanhhm/research-idea/plasma/parallel_processing/communicators/sequential.py:52: UserWarning: this class is deprecated, use Sequential instead
  warn('this class is deprecated, use Sequential instead')


*-[ProcessQueue(name=None, runner=2)]

  |-(compute2:<function compute at 0x7f318c125800>)

    |-[ProcessQueue(name=None, runner=1)]

      |-(printer:<function print_pass at 0x7f318c173920>)

        |-[TransferQueue(name=None, runner=1)]

          |-(aggregator:Aggregator)

            |-[ThreadQueue((name=None, runner=1)]-*

In [5]:
caller.run()

*-[ProcessQueue(name=compute2, runner=2)]

  |-(compute2:<function compute at 0x7f318c125800>)

    |-[ProcessQueue(name=printer, runner=1)]

      |-(printer:<function print_pass at 0x7f318c173920>)

        |-[TransferQueue(name=aggregator, runner=1)]

          |-(aggregator:Aggregator)

            |-[ThreadQueue((name=None, runner=1)]-*

In [ ]:
with caller:
    for i in range(20):
        caller.put(i)
    
    print(caller.aggregator.wait())

In [ ]:
caller.print = print

In [ ]:
caller.registerIOs(print=pp.queues.ThreadQueue(), outputs=pp.queues.ThreadQueue())